<a href="https://colab.research.google.com/github/micheleintrevado/ML4MDE/blob/main/CodeCommentClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf
from google.colab import drive
from os.path import join
from zipfile import ZipFile
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense
import nltk
import re

# trying to cowork with colab

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
drive.mount('/content/drive/',force_remount=False)

DATA_PATH = '/content/drive/My Drive/'

dataset_file = join(DATA_PATH,'dataset.zip')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
dataset_file

'/content/drive/My Drive/dataset.zip'

In [ ]:
zip_file = ZipFile(dataset_file)
zip_file

<zipfile.ZipFile filename='/content/drive/My Drive/dataset.zip' mode='r'>

In [ ]:
dfs = {text_file.filename: pd.read_csv(zip_file.open(text_file.filename)) for text_file in zip_file.infolist() if text_file.filename.endswith('.csv')}
dfs

{'sample_submission.csv':          ID  Computer Science  Physics  Mathematics  Statistics  \
 0     20973                 0        0            0           0   
 1     20974                 0        0            0           0   
 2     20975                 0        0            0           0   
 3     20976                 0        0            0           0   
 4     20977                 0        0            0           0   
 ...     ...               ...      ...          ...         ...   
 8984  29957                 0        0            0           0   
 8985  29958                 0        0            0           0   
 8986  29959                 0        0            0           0   
 8987  29960                 0        0            0           0   
 8988  29961                 0        0            0           0   
 
       Quantitative Biology  Quantitative Finance  
 0                        0                     0  
 1                        0                     0  
 

##### Since the "test.sv" file does not provide any labels, we decided to use only the "train.csv" file

In [ ]:
dataset = dfs['train.csv'].drop(columns=['ID'])
dataset

,TITLE,ABSTRACT,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance
0,Reconstructing Subject-Specific Effect Maps,Predictive models allow subject-specific inf...,1,0,0,0,0,0
1,Rotation Invariance Neural Network,Rotation invariance and translation invarian...,1,0,0,0,0,0
2,Spherical polyharmonics and Poisson kernels fo...,We introduce and develop the notion of spher...,0,0,1,0,0,0
3,A finite element approximation for the stochas...,The stochastic Landau--Lifshitz--Gilbert (LL...,0,0,1,0,0,0
4,Comparative study of Discrete Wavelet Transfor...,Fourier-transform infra-red (FTIR) spectra o...,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...
20967,Contemporary machine learning: a guide for pra...,Machine learning is finding increasingly bro...,1,1,0,0,0,0
20968,Uniform diamond coatings on WC-Co hard alloy c...,Polycrystalline diamond coatings have been g...,0,1,0,0,0,0
20969,Analysing Soccer Games with Clustering and Con...,We present a new approach for identifying si...,1,0,0,0,0,0
20970,On the Efficient Simulation of the Left-Tail o...,The sum of Log-normal variates is encountere...,0,0,1,1,0,0


In [ ]:
x = dataset[['TITLE','ABSTRACT']]
y = dataset.drop(columns=['TITLE','ABSTRACT'], axis = 1)
x, y

(                                                   TITLE  \
 0            Reconstructing Subject-Specific Effect Maps   
 1                     Rotation Invariance Neural Network   
 2      Spherical polyharmonics and Poisson kernels fo...   
 3      A finite element approximation for the stochas...   
 4      Comparative study of Discrete Wavelet Transfor...   
 ...                                                  ...   
 20967  Contemporary machine learning: a guide for pra...   
 20968  Uniform diamond coatings on WC-Co hard alloy c...   
 20969  Analysing Soccer Games with Clustering and Con...   
 20970  On the Efficient Simulation of the Left-Tail o...   
 20971   Why optional stopping is a problem for Bayesians   
 
                                                 ABSTRACT  
 0        Predictive models allow subject-specific inf...  
 1        Rotation invariance and translation invarian...  
 2        We introduce and develop the notion of spher...  
 3        The stochastic L

In [ ]:
train_data, test_data, train_labels, test_labels = train_test_split(x, y, test_size=0.2, random_state=42)
train_data, test_data, train_labels, test_labels

(                                                   TITLE  \
 16466  Reverse Quantum Annealing Approach to Portfoli...   
 4021                Arithmetic properties of polynomials   
 17248  LATTE: Application Oriented Social Network Emb...   
 17239  Full Momentum and Energy Resolved Spectral Fun...   
 2966   Collisional Dynamics of Solitons in the Couple...   
 ...                                                  ...   
 11284     Tensor Methods for Nonlinear Matrix Completion   
 11964  Numerical Simulations of Collisional Cascades ...   
 5390   Possible evidence for spin-transfer torque ind...   
 860               Common Knowledge in a Logic of Gossips   
 15795  Trust-Based Collaborative Filtering: Tackling ...   
 
                                                 ABSTRACT  
 16466    We investigate a hybrid quantum-classical so...  
 4021     In this paper, first, we prove that the Diop...  
 17248    In recent years, many research works propose...  
 17239    The single-parti

In [ ]:
contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not",
                           "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not",
                           "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",
                           "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would",
                           "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",
                           "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam",
                           "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have",
                           "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock",
                           "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have",
                           "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is",
                           "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as",
                           "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",
                           "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have",
                           "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have",
                           "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are",
                           "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",
                           "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is",
                           "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have",
                           "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",
                           "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all",
                           "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",
                           "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have",
                           "you're": "you are", "you've": "you have"}

In [ ]:
stop_words = set(nltk.corpus.stopwords.words('english'))

def text_cleaner(text,num):
    newString = text.lower() # convert to lower case
    newString = re.sub('"','', newString) # convert the double apex " into single apex '
    newString = ' '.join([contraction_mapping[t] if t in contraction_mapping else t for t in newString.split(" ")]) # substitute the contract form with the extended one
    newString = re.sub(r"'s\b","",newString)  # remove the 's contraction
    newString = re.sub("[^a-zA-Z]", " ", newString) # remove non aplhabetic characters

    if(num==0):
        tokens = [w for w in newString.split() if not w in stop_words] # remove the stopwords
    else:
        tokens=newString.split()
    long_words=[]
    for i in tokens:
        if len(i)>1:                                                 #removing short word
            long_words.append(i)
    return (" ".join(long_words)).strip()

In [ ]:
#call the function
cleaned_body = []
for t in train_data['ABSTRACT']:
    cleaned_body.append(text_cleaner(t,0))
cleaned_body

['investigate hybrid quantum classical solution method mean variance portfolio optimization problems starting real financial data statistics following principles modern portfolio theory generate parametrized samples portfolio optimization problems related quadratic binary optimization forms programmable analog wave quantum annealer instances also solvable industry established genetic algorithm approach use classical benchmark investigate several options run quantum computation optimally ultimately discovering best results terms expected time solution function number variables hardest instances set obtained seeding quantum annealer solution candidate found greedy local search performing reverse annealing protocol optimized reverse annealing protocol found times faster corresponding forward quantum annealing average',
 'paper first prove diophantine system infinitely many integer solutions nonzero integers equiv pmod second show diophantine system integer parametric solution nonzero inte

In [ ]:
#call the function
cleaned_title = []
for t in train_data['TITLE']:
    cleaned_title.append(text_cleaner(t,1))

cleaned_title

['reverse quantum annealing approach to portfolio optimization problems',
 'arithmetic properties of polynomials',
 'latte application oriented social network embedding',
 'full momentum and energy resolved spectral function of electronic system',
 'collisional dynamics of solitons in the coupled pt symmetric nonlocal nonlinear schrodinger equations',
 'algebraic surfaces with zero dimensional cohomology support locus',
 'special lagrangian and deformed hermitian yang mills on tropical manifold',
 'exponentially convergent data assimilation algorithm for navier stokes equations',
 'adversarial connective exploiting networks for implicit discourse relation classification',
 'new torsion pendulum for gravitational reference sensor technology development',
 'fisher gan',
 'on line tracing of xacml based policy coverage criteria',
 'ip based traffic recovery an optimal approach using sdn application for data center network',
 'limits on the anomalous speed of gravitational waves from binar

In [ ]:
cleaned_dataset = pd.DataFrame()
cleaned_dataset['cleaned_title'] = cleaned_title
cleaned_dataset['cleaned_body'] = cleaned_body
cleaned_dataset

,cleaned_title,cleaned_body
0,reverse quantum annealing approach to portfoli...,investigate hybrid quantum classical solution ...
1,arithmetic properties of polynomials,paper first prove diophantine system infinitel...
2,latte application oriented social network embe...,recent years many research works propose embed...
3,full momentum and energy resolved spectral fun...,single particle spectral function measures den...
4,collisional dynamics of solitons in the couple...,investigate focusing coupled pt symmetric nonl...
...,...,...
16772,tensor methods for nonlinear matrix completion,low rank matrix completion lrmc problem low ra...
16773,numerical simulations of collisional cascades ...,consider long term collisional dynamical evolu...
16774,possible evidence for spin transfer torque ind...,cooper pairs superconductors normally spin sin...
16775,common knowledge in logic of gossips,gossip protocols aim arriving means point poin...


# VEDERE SE TOGLIERE BODY E TITOLI TROPPO LUNGHI

In [ ]:
#prepare a tokenizer for reviews on training data
x_tokenizer = Tokenizer()
x_tokenizer.fit_on_texts(list(cleaned_dataset['cleaned_body']))

In [ ]:
#max_body_len = 100

#convert text sequences into integer sequences
x_tr_seq    =   x_tokenizer.texts_to_sequences(cleaned_dataset['cleaned_body'])
#x_val_seq   =   x_tokenizer.texts_to_sequences(x_val)

#padding zero upto maximum length
#x_tr    =   pad_sequences(x_tr_seq,  maxlen=max_body_len, padding='post')
x_tr    =   pad_sequences(x_tr_seq, padding='post')

#x_val   =   pad_sequences(x_val_seq, maxlen=max_body_len, padding='post')

In [ ]:
max_words = 1000
max_len = 100

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(train_data)

train_sequences = tokenizer.texts_to_sequences(train_data)
test_sequences = tokenizer.texts_to_sequences(test_data)

train_data_padded = pad_sequences(train_sequences, maxlen=max_len)
test_data_padded = pad_sequences(test_sequences, maxlen=max_len)

model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=50, input_length=max_len))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(6, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(train_data_padded, train_labels, epochs=5, batch_size=32, validation_split=0.2)

Epoch 1/5
1/1 [==============================] - 1s 1s/step - loss: 0.7013 - accuracy: 0.0000e+00 - val_loss: 0.6928 - val_accuracy: 0.0000e+00
Epoch 2/5
1/1 [==============================] - 0s 40ms/step - loss: 0.6280 - accuracy: 1.0000 - val_loss: 0.6984 - val_accuracy: 0.0000e+00
Epoch 3/5
1/1 [==============================] - 0s 53ms/step - loss: 0.5763 - accuracy: 1.0000 - val_loss: 0.7078 - val_accuracy: 0.0000e+00
Epoch 4/5
1/1 [==============================] - 0s 44ms/step - loss: 0.5263 - accuracy: 1.0000 - val_loss: 0.7192 - val_accuracy: 0.0000e+00
Epoch 5/5
1/1 [==============================] - 0s 59ms/step - loss: 0.4778 - accuracy: 1.0000 - val_loss: 0.7320 - val_accuracy: 0.0000e+00


In [ ]:
train_sequences

[[1], [2]]

In [ ]:
predictions = model.predict(test_data_padded)
predictions

1/1 [==============================] - 0s 124ms/step


array([[0.33845523, 0.24276125, 0.30483815, 0.31867838, 0.56381947,
        0.72852427],
       [0.33868194, 0.24610023, 0.30762494, 0.32037255, 0.5629647 ,
        0.7283305 ]], dtype=float32)